In [1]:
import pandas as pd


In [2]:
df= pd.read_csv("data/rat_weight_water_intake.csv")

In [7]:
df['gender'] = df['rat_id'].str.lower().apply(lambda x: 'male' if 'male' in x and 'female' not in x else 'female')

In [11]:
df['unique_rat_id'] = 'G' + df['water_group'].str.extract(r'Group (\d+)')[0] + '_' + df['gender'].str[0].str.upper() + df['rat_number'].astype(str)

In [12]:
df.head()

,water_group,rat_id,rat_number,gender,week,weekly_feed_intake,initial_body_weight,weekly_weight_gain,unique_rat_id
0,Group 1 (RO water with < 20 TDS),Group 1 Male Rats,1,male,1,10.000,59.0,51.0,G1_M1
1,Group 1 (RO water with < 20 TDS),Group 1 Male Rats,1,male,2,12.000,NaN,43.0,G1_M1
2,Group 1 (RO water with < 20 TDS),Group 1 Male Rats,1,male,3,10.800,NaN,29.0,G1_M1
3,Group 1 (RO water with < 20 TDS),Group 1 Male Rats,1,male,4,11.880,NaN,20.0,G1_M1
4,Group 1 (RO water with < 20 TDS),Group 1 Male Rats,1,male,5,16.619,NaN,18.0,G1_M1


In [15]:
# First, forward fill the initial_body_weight for each rat
df['initial_body_weight'] = df.groupby('unique_rat_id')['initial_body_weight'].ffill()

# Calculate cumulative weight gain for each rat
df['cumulative_weight_gain'] = df.groupby('unique_rat_id')['weekly_weight_gain'].cumsum()

# Calculate current body weight (initial + cumulative gain)
df['current_body_weight'] = df['initial_body_weight'] + df['cumulative_weight_gain']

# Create final_body_weight column (only populated at week 15)
df['final_body_weight'] = df.apply(lambda row: row['current_body_weight'] if row['week'] == 15 else None, axis=1)

# If you want to fill final_body_weight for all weeks of each rat (propagate week 15 value)
df['final_body_weight'] = df.groupby('unique_rat_id')['final_body_weight'].transform(lambda x: x.ffill().bfill())

In [ ]:
# df.to_csv('/Users/praharshita/Desktop/Projects/weight-vs-water-type/data/processed_rat_data.csv', index=False)